<a href="https://colab.research.google.com/github/Dinma-gif/No_show_appointment-Analysis/blob/main/Data_engineering1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install faker
!pip install elasticsearch


#Inserting and extracting NoSQL database data in Python

Relational databases may be what you think of when you hear the term database, but
there are several other types of databases, such as columnar, key-value, and time-series.
In this section, you will learn how to work with Elasticsearch, which is a NoSQL database.
NoSQL is a generic term referring to databases that do not store data in rows and columns.
NoSQL databases often store their data as JSON documents and use a query language other
than SQL. The next section will teach you how to load data into Elasticsearch.

## Inserting data using helpers
Using the bulk methis, you can insert many documents at a time. The process is similar to inserting a single record, except that you will generate all the data, then insert it.

In [4]:
from faker import Faker
from elasticsearch import Elasticsearch, helpers

# Initialize Faker Instance
fake = Faker()

# Initialize the Elasticsearch client
es = Elasticsearch(#connection#)

# Prepare the list of actions for the bulk insert
#The data needs to be an array of JSON objects. You must identify the index and the type. Underscores in the names are used for Elasticsearch fields. The _source field is where you would put the JSON document that you want to insert in the database.

actions = [
    {
        "_op_type": "index",  # Action type, you can use "index" or "create"
        "_index": "fake_data",  # Your index name
        "_id": fake.uuid4(),  # Unique ID for each document
        "_source": {
            "name": fake.name(),
            "street": fake.street_address(),
            "city": fake.city(),
            "state": fake.state(),
            "zip": fake.zipcode()
        }
    }
    for _ in range(998)  # Loop 998 times to create 998 documents
]

# Perform the bulk insert operation
success, failed = helpers.bulk(es, actions)

# Print the result
print(f"Successfully indexed {success} documents.")
print(f"Failed to index {failed} documents.")


Successfully indexed 998 documents.
Failed to index [] documents.


Create a JSON object to send to Elasttic. The object is a query, using the match_all search.

In [6]:
doc={"query":{"match_all":{}}}

Now, you can call the bulk method and pass it the elasticsearch instance and the array of data. You can print the results to check that it worked.

In [7]:
res=es.search(index="fake_data",body=doc)
print(res['hits']['hits'])

[{'_index': 'fake_data', '_id': 'cqheTJQBpVBMzcXIuj6T', '_score': 1.0, '_source': {'name': 'Christian Holloway', 'street': '859 Kimberly Ridge Suite 223', 'city': 'South Dannyhaven', 'state': 'Hawaii', 'zip': '78523'}}, {'_index': 'fake_data', '_id': 'c06b35fe-0ecf-40a5-9184-8aa98f32625f', '_score': 1.0, '_source': {'name': 'Mrs. Rachel Smith', 'street': '087 Mack Coves', 'city': 'Lindaburgh', 'state': 'Vermont', 'zip': '19482'}}, {'_index': 'fake_data', '_id': 'ce616cea-733e-4888-915c-08bd896ee509', '_score': 1.0, '_source': {'name': 'John Hendrix', 'street': '81666 Amber Prairie Apt. 402', 'city': 'North Ryan', 'state': 'Minnesota', 'zip': '42677'}}, {'_index': 'fake_data', '_id': '16b859c9-0698-4881-8621-db5bbb30e6ca', '_score': 1.0, '_source': {'name': 'Angela Vargas', 'street': '936 Cynthia Mill', 'city': 'Mooreshire', 'state': 'Alaska', 'zip': '99163'}}, {'_index': 'fake_data', '_id': '0512d0ae-ed2e-425e-8708-98c340727533', '_score': 1.0, '_source': {'name': 'Erika Pineda', 'stre

In [8]:
for doc in res['hits']['hits']:
    print(doc['_source'])

{'name': 'Christian Holloway', 'street': '859 Kimberly Ridge Suite 223', 'city': 'South Dannyhaven', 'state': 'Hawaii', 'zip': '78523'}
{'name': 'Mrs. Rachel Smith', 'street': '087 Mack Coves', 'city': 'Lindaburgh', 'state': 'Vermont', 'zip': '19482'}
{'name': 'John Hendrix', 'street': '81666 Amber Prairie Apt. 402', 'city': 'North Ryan', 'state': 'Minnesota', 'zip': '42677'}
{'name': 'Angela Vargas', 'street': '936 Cynthia Mill', 'city': 'Mooreshire', 'state': 'Alaska', 'zip': '99163'}
{'name': 'Erika Pineda', 'street': '09238 Simon Well', 'city': 'New Glennhaven', 'state': 'Hawaii', 'zip': '75814'}
{'name': 'Jessica Zimmerman', 'street': '2805 Rodriguez Dale', 'city': 'Jenniferton', 'state': 'South Dakota', 'zip': '20231'}
{'name': 'Cameron Bowers', 'street': '2318 Rachael Ranch', 'city': 'Lake Aprilhaven', 'state': 'Minnesota', 'zip': '43280'}
{'name': 'Regina Garcia', 'street': '674 Clark Green', 'city': 'East Leslie', 'state': 'Iowa', 'zip': '44494'}
{'name': 'Thomas Maynard', 'st

You can load the results of the query into a pandas DataFrame- it is JSON, and you learned how to read JSON ealier. The import json_normalize from the pandas json library, and use it(json_normalize) on the JSON results


In [9]:
from pandas import json_normalize
import pandas as pd
df=json_normalize(res['hits']['hits'])
df.head()

,_index,_id,_score,_source.name,_source.street,_source.city,_source.state,_source.zip
0,fake_data,cqheTJQBpVBMzcXIuj6T,1.0,Christian Holloway,859 Kimberly Ridge Suite 223,South Dannyhaven,Hawaii,78523
1,fake_data,c06b35fe-0ecf-40a5-9184-8aa98f32625f,1.0,Mrs. Rachel Smith,087 Mack Coves,Lindaburgh,Vermont,19482
2,fake_data,ce616cea-733e-4888-915c-08bd896ee509,1.0,John Hendrix,81666 Amber Prairie Apt. 402,North Ryan,Minnesota,42677
3,fake_data,16b859c9-0698-4881-8621-db5bbb30e6ca,1.0,Angela Vargas,936 Cynthia Mill,Mooreshire,Alaska,99163
4,fake_data,0512d0ae-ed2e-425e-8708-98c340727533,1.0,Erika Pineda,09238 Simon Well,New Glennhaven,Hawaii,75814


In [10]:
doc={"query":{"match":{"name":"Paul Goodman"}}}
res=es.search(index="fake_data",body=doc)
print(res['hits']['hits'][0]['_source'])

{'name': 'Paul Goodman', 'street': '547 Michelle View', 'city': 'East Williamville', 'state': 'Tennessee', 'zip': '75407'}


In [11]:
res=es.search(index='fake_data',q='name:Paul Goodman')
print(res['hits']['hits'][0]['_source'])

{'name': 'Paul Goodman', 'street': '547 Michelle View', 'city': 'East Williamville', 'state': 'Tennessee', 'zip': '75407'}


In [12]:
#Get city Jamesberg -Returns Janmesberg and Lake Jamesberg
doc={"query":{"match":{"city":"Lindaburgh"}}}
res=es.search(index="fake_data",body=doc)
print(res['hits']['hits'][0]['_source'])

{'name': 'Mrs. Rachel Smith', 'street': '087 Mack Coves', 'city': 'Lindaburgh', 'state': 'Vermont', 'zip': '19482'}


In [3]:
!pip install apache-airflow
import os
import os

# Set Airflow home directory (this can be changed if needed)
os.environ['AIRFLOW_HOME'] = '/content/airflow'

# Optional: Set a connection string for the database (if you're using something other than SQLite)
# Here we are using SQLite for simplicity
os.environ['AIRFLOW__CORE__SQL_ALCHEMY_CONN'] = 'sqlite:////content/airflow/airflow.db'

!pip install psycopg2

!airflow db init

DB: sqlite:////content/airflow/airflow.db
[2025-01-10T15:57:36.592+0000] {migration.py:207} INFO - Context impl SQLiteImpl.
[2025-01-10T15:57:36.593+0000] {migration.py:210} INFO - Will assume non-transactional DDL.
[2025-01-10T15:57:36.781+0000] {migration.py:207} INFO - Context impl SQLiteImpl.
[2025-01-10T15:57:36.782+0000] {migration.py:210} INFO - Will assume non-transactional DDL.
[2025-01-10T15:57:36.784+0000] {migration.py:207} INFO - Context impl SQLiteImpl.
[2025-01-10T15:57:36.784+0000] {migration.py:210} INFO - Will assume non-transactional DDL.
[2025-01-10T15:57:36.785+0000] {db.py:1675} INFO - Creating tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
WARNI [airflow.models.crypto] empty cryptography key - values will not be stored encrypted.
Initialization done


In [1]:
!pip install --upgrade apache-airflow sqlalchemy



  Using cached SQLAlchemy-2.0.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)


###Building  data pipelines in Apache Airflow

I will combine two Python Operators to extract data from postgreSQL, save it as a CSV file, then read it in and write it to an Elasticsearch index. For this tasks, I will import the pandas, pscycopgq, and elasticsearch libraries.

In [4]:
import datetime as dt
from datetime import timedelta

from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator

import pandas as pd
import psycopg2 as db
from elasticsearch import Elasticsearch

#Next, I will specify the arguments for my DAG. Remember, the start time should be a day behind if you schedule the task to run daily.
default_args= {
    'owner':  'dinma',
    'start_date':  dt.datetime(2023,  1,  9),
    'retries': 1,
    'retry_delay': dt.timedelta(minutes=5),
    }

#Here I will create the python functions, one to connect to postgreSQL and one to connect to  Elasticsearch.

def queryPostgresql():
    conn_string = "dbname= 'public' host='host' user='***' password='***'"
    conn = db.connect(conn_string)
    df = pd.read_sql_query("SELECT name, city FROM users", conn)
    df.to_csv('postgresqldata.csv')
    print("------Data saved--------")


def insertElasticsearch():
    es = Elasticsearch(#connection)
    df = pd.read_csv('postgresqldata.csv')
    for i, r in df.iterrows():
        doc = r.to_json()
        res = es.index(index="frompostgresql", body=doc)
        print(res)

# Now I will create two Python operators , one to get the data from postgreSQL and the other to insert data into Elasticsearch. The getData task will be upstream and the insertData task will be downstream.

with DAG('MyDBdag',
                   default_args=default_args,
                   schedule_interval=timedelta(minutes=5),
                                                   # '0 * * * *',
                 )  as dag:
    getData=PythonOperator(task_id='QueryPostgresSQL',
                                   python_callable=queryPostgresql)
    insertData = PythonOperator(
        task_id='InsertDataElasticsearch',
        python_callable=insertElasticsearch
    )

    getData >> insertData



[2025-01-10T15:57:47.633+0000] {utils.py:162} INFO - NumExpr defaulting to 2 threads.


<ipython-input-4-dbcd39b90c7f>:40 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.

In [8]:
!airflow webserver -p 8081 -H 0.0.0.0 &






  ____________       _____________
 ____    |__( )_________  __/__  /________      __
____  /| |_  /__  ___/_  /_ __  /_  __ \_ | /| / /
___  ___ |  / _  /   _  __/ _  / / /_/ /_ |/ |/ /
 _/_/  |_/_/  /_/    /_/    /_/  \____/____/|__/
Running the Gunicorn Server with:
Workers: 4 sync
Host: 0.0.0.0:8081
Timeout: 120
Logfiles: - -
Access Logformat: 
/usr/local/lib/python3.10/dist-packages/flask_limiter/extension.py:333 UserWarning: Using the in-memory storage for tracking rate limits as no storage was explicitly specified. This is not recommended for production use. See: https://flask-limiter.readthedocs.io#configuring-a-storage-backend for documentation about configuring the storage backend.
[2025-01-10T16:11:37.593+0000] {utils.py:162} INFO - NumExpr defaulting to 2 threads.
[2025-01-10 16:11:39 +0000] [20419] [INFO] Starting gunicorn 23.0.0
[2025-01-10 16:11:39 +0000] [20419] [INFO] Listening at: http://0.0.0.0:8081 (20419)
[2025-01-10 16:11:39 +0000] [20419] [INFO] Using worker: syn